In [2]:
using BenchmarkTools: @btime
import LinearAlgebra as la

In [3]:
mutable struct Fast_Det
    const D::Matrix{Float64}
    const D_inv::Matrix{Float64}
    const n::Int64
    det::Float64

    
    function Fast_Det(D)
        D_inv = la.inv(D)
        det = la.det(D)
        n = size(D)[1]
        return new(copy(D), D_inv, n, det)
    end
end

In [4]:
function det(fd::Fast_Det)
    return fd.det
end

function det_new_row(fd::Fast_Det, new_row::Vector{Float64}, i)
    (; D_inv, n, det) = fd
    R = 0.0
    for j in 1:n
        R += new_row[j] * D_inv[j, i]
    end
    return det * R
end

function ratio_new_old_det(fd::Fast_Det, new_row::Vector{Float64}, i)
    (; D_inv, n) = fd
    R = 0.0
    for j in 1:n
        R += new_row[j] * D_inv[j, i]
    end
    return R
end

function change_row!(fd, new_row, i)
    (; D, D_inv, n) = fd
    R = 0.0
    for j in 1:n
        R += new_row[j] * D_inv[j, i]
    end
    fd.det = fd.det * R

    for j in 1:n
        if j != i
            S = 0.0
            for l in 1:n
                S += new_row[l] * D_inv[l, j]
            end
        
            for k in 1:n
                D_inv[k, j] = D_inv[k, j] - S / R * D_inv[k, i]
            end
        end
    end

    for k in 1:n
        D[i, k] = new_row[k]
        D_inv[k, i] /= R
    end
    
    return fd
end

change_row! (generic function with 1 method)

In [5]:
n = 20
D = rand(n, n);
fd = Fast_Det(D);

i = 1
new_row = rand(n);

In [6]:
new_D = copy(D)
new_D[i, :] .= new_row

old_det = la.det(D)
new_det = la.det(new_D)
ratio_new_old_det(fd, new_row, i) - new_det / old_det

8.881784197001252e-16

In [13]:
@btime la.det(new_D)

  4.314 μs (3 allocations: 3.48 KiB)


-0.015579687659046023

In [14]:
@btime det_new_row(fd, new_row, i)

  51.724 ns (1 allocation: 16 bytes)


-0.01557968765904603

In [9]:
@btime change_row!(fd, new_row, i);

  608.621 ns (0 allocations: 0 bytes)


In [15]:
4314/51.724

83.40422241125977